<a href="https://colab.research.google.com/github/kmayutrisna/SA_IMDB/blob/main/Bert_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout

In [3]:
#Get the dataset from gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Display top of data
df =pd.read_csv("drive/My Drive/data/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
# Define the split percentages (adjust as needed)
train_split = 0.8
val_split = 0.1
test_split = 0.1

In [6]:
# Shuffle the dataset
df = df.sample(frac=1).reset_index(drop=True)

In [7]:
# Calculate the sizes of each split
train_size = int(train_split * len(df))
val_size = int(val_split * len(df))
test_size = len(df) - train_size - val_size

In [8]:
# Split the dataset
train_data_df = df[:train_size]
val_data_df = df[train_size:train_size + val_size]
test_data_df = df[train_size + val_size:]

In [9]:
# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [10]:
# Define a function for tokenizing and padding sequences
def tokenize_and_pad(review, sentiment):
    def tokenization_fn(review, sentiment):
        tokens = tokenizer.encode_plus(
            review.numpy().decode('utf-8'),
            add_special_tokens=True,
            max_length=128,
            padding='max_length',
            return_attention_mask=True,
            truncation=True
        )
        return tokens['input_ids'], tokens['attention_mask'], sentiment

    input_ids, attention_mask, sentiment = tf.py_function(tokenization_fn, inp=[review, sentiment], Tout=[tf.int32, tf.int32, tf.string])
    input_ids.set_shape([128])
    attention_mask.set_shape([128])

    # Convert sentiment labels from strings to integers
    sentiment = tf.cond(tf.strings.regex_full_match(sentiment, 'positive'), lambda: tf.constant(1, dtype=tf.int32), lambda: tf.constant(0, dtype=tf.int32))

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }, sentiment

In [11]:
# Tokenize and pad the sequences
train_data = tf.data.Dataset.from_tensor_slices((train_data_df['review'], train_data_df['sentiment']))
train_data = train_data.map(tokenize_and_pad)

val_data = tf.data.Dataset.from_tensor_slices((val_data_df['review'], val_data_df['sentiment']))
val_data = val_data.map(tokenize_and_pad)

test_data = tf.data.Dataset.from_tensor_slices((test_data_df['review'], test_data_df['sentiment']))
test_data = test_data.map(tokenize_and_pad)

In [12]:
# Step 4: Create the model

# Load the BERT model
bert_model = TFBertModel.from_pretrained("bert-base-uncased")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [13]:
# Freeze BERT layers
for layer in bert_model.layers:
    layer.trainable = False

In [14]:
# Define input layers
from tensorflow.keras.layers import Input
input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')


In [15]:
# Get BERT embeddings
bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]

In [16]:
# Add GRU layer
from tensorflow.keras.layers import Input, GRU
gru_output = GRU(128)(bert_output)

In [18]:
# Add Dense layer for classification
output = Dense(1, activation='sigmoid')(gru_output)

In [19]:
# Create the custom model
from tensorflow.keras.models import Model
model = Model(inputs=[input_ids, attention_mask], outputs=output)

In [20]:
# Step 5: Compile and train the model

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
# Train the model
model.fit(train_data.batch(32), epochs=10)


Epoch 1/10
1250/1250 [==============================] - 445s 340ms/step - loss: 0.3725 - accuracy: 0.8306
Epoch 2/10
1250/1250 [==============================] - 425s 340ms/step - loss: 0.2939 - accuracy: 0.8727
Epoch 3/10
1250/1250 [==============================] - 426s 341ms/step - loss: 0.2588 - accuracy: 0.8909
Epoch 4/10
1250/1250 [==============================] - 422s 338ms/step - loss: 0.2244 - accuracy: 0.9075
Epoch 5/10
1250/1250 [==============================] - 423s 338ms/step - loss: 0.1918 - accuracy: 0.9225
Epoch 6/10
1250/1250 [==============================] - 419s 335ms/step - loss: 0.1627 - accuracy: 0.9365
Epoch 7/10
1250/1250 [==============================] - 420s 335ms/step - loss: 0.1442 - accuracy: 0.9433
Epoch 8/10
1250/1250 [==============================] - 415s 332ms/step - loss: 0.1258 - accuracy: 0.9495
Epoch 9/10
1250/1250 [==============================] - 416s 332ms/step - loss: 0.1170 - accuracy: 0.9539
Epoch 10/10
1250/1250 [=======================

In [22]:
# Step 6: Evaluate the model on test data

# Evaluate on the test dataset
loss, accuracy = model.evaluate(test_data.batch(32))

# Print accuracy
print('Accuracy:', accuracy)

157/157 [==============================] - 67s 385ms/step - loss: 0.4242 - accuracy: 0.8712
Accuracy: 0.8712000250816345


In [23]:
# Predict on the test dataset
y_pred = model.predict(test_data.batch(32))

# Convert predictions to binary labels (0 or 1)
y_pred_binary = (y_pred > 0.5).astype(int)

# Get true labels from the test dataset
y_true = [sentiment.numpy() for _, sentiment in test_data]

157/157 [==============================] - 55s 327ms/step


In [24]:
# Calculate precision, recall, and F1 score using TensorFlow metrics
precision = tf.keras.metrics.Precision()(y_true, y_pred_binary).numpy()
recall = tf.keras.metrics.Recall()(y_true, y_pred_binary).numpy()
f1_score = 2 * (precision * recall) / (precision + recall)

In [25]:
# Print precision, recall, and F1 score
print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1_score)

Precision: 0.8949153
Recall: 0.84210527
F1 Score: 0.8677075246596296
